<a href="https://colab.research.google.com/github/DiegoPincheiraIb/Experiencias_EL5206/blob/master/Experiencia_2/Experiencia_2_EL5206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiencia numero 2: Luckas Diaz - Diego Pincheira

# Configuraciones Preliminares

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import color, io, exposure
import os
# pip install natsort # Se debe instalar natsorted para ejecutar el código
from natsort import natsorted

Cargar los archivos *ojos_gal.rar*, *ojos_impostor.rar* y ojos_test.rar a colab

In [2]:
from google.colab import files
uploaded = files.upload()

Saving ojos_gal.rar to ojos_gal.rar
Saving ojos_impostor.rar to ojos_impostor.rar
Saving ojos_test.rar to ojos_test.rar


Se descomprimen los archivos

In [3]:
!unrar x ojos_gal.rar
!unrar x ojos_impostor.rar
!unrar x ojos_test.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from ojos_gal.rar

Creating    ojos_gal                                                  OK
Extracting  ojos_gal/I10_gal.jpg                                           9%  OK 
Extracting  ojos_gal/I1_gal.jpg                                           19%  OK 
Extracting  ojos_gal/I2_gal.jpg                                           31%  OK 
Extracting  ojos_gal/I3_gal.jpg                                           42%  OK 
Extracting  ojos_gal/I4_gal.jpg                                           50%  OK 
Extracting  ojos_gal/I5_gal.jpg                                           60%  OK 
Extracting  ojos_gal/I6_gal.jpg                                           71%  OK 
Extracting  ojos_gal/I7_gal.jpg                                           79%  OK 
Extracting  ojos_gal/I8_gal.jpg                                           90%  OK 


In [4]:
!ls

ojos_gal      ojos_impostor	 ojos_test	sample_data
ojos_gal.rar  ojos_impostor.rar  ojos_test.rar


Se definen los path de las imagenes

In [5]:
Str_Gal = 'ojos_gal/'
Str_Impostor = 'ojos_impostor/ojos_impostor/'
Str_Test = 'ojos_test/'
Listado_Gal = os.listdir(Str_Gal)
Listado_Gal = natsorted(Listado_Gal)
Listado_Impostor = os.listdir(Str_Impostor)
Listado_Impostor = natsorted(Listado_Impostor)
Listado_Test = os.listdir(Str_Test)
Listado_Test = natsorted(Listado_Test)

Convertir a escala de grises y equalizar las imagenes

In [6]:
Listado_Gal_Gris = []
for imagen in Listado_Gal:
  Listado_Gal_Gris.append(exposure.equalize_adapthist(color.rgb2gray(io.imread(Str_Gal + imagen))))

Listado_Impostor_Gris = []
for imagen in Listado_Impostor:
  Listado_Impostor_Gris.append(exposure.equalize_adapthist(color.rgb2gray(io.imread(Str_Impostor + imagen))))

Listado_Test_Gris = []
for imagen in Listado_Test:
  Listado_Test_Gris.append(exposure.equalize_adapthist(color.rgb2gray(io.imread(Str_Test + imagen))))

# **Desarrollo P1**

Algoritmo LBP


In [7]:
def LBP(ImagenGris_Matriz):
  Matriz_Final_Copia = np.copy(ImagenGris_Matriz)
  Mascara_Valores = np.array([[128., 64., 32.],
                  [1., 0., 1.],
                  [2., 4., 8.]])
  for row in range(1,len(ImagenGris_Matriz)-1):
    for column in range(1,len(ImagenGris_Matriz[0])-1):
      mask = np.zeros((3,3))
      mask[1,1] = ImagenGris_Matriz[row,column]
      for fila in range(-1,2):
        for columna in range(-1,2):
          if fila != 0 and columna !=0:                      
            if ImagenGris_Matriz[row+fila,column+columna] > mask[1,1]:
              mask[fila, columna] = 1
      Numero_Entero = np.sum(mask * Mascara_Valores)
      Matriz_Final_Copia[row, column] = Numero_Entero

  Matriz_Final_Copia_Copia = np.delete(Matriz_Final_Copia, -1, 0) # Elimina la última fila
  Matriz_Final_Copia = np.delete(Matriz_Final_Copia_Copia, -1, 1) # Elimina la última columna
  Matriz_Final_Copia_Copia = np.delete(Matriz_Final_Copia, 0, 0) # Elimina la primera fila
  Matriz_Final_Copia = np.delete(Matriz_Final_Copia_Copia, 0, 1) # Elimina la primera columna
  return Matriz_Final_Copia
  
def Imagen_Plot(Matriz_Obj):
  plt.imshow(Matriz_Obj, cmap='gray')
  plt.show()

Aplicar LBP a todas la imagenes y guardarlas en una lista



In [8]:
Gal_LBP_List = []
Impostor_LBP_List = []
Test_LBP_List = []

for pic in Listado_Gal_Gris:
  Gal_LBP_List.append(LBP(pic))

for pic in Listado_Impostor_Gris:
  Impostor_LBP_List.append(LBP(pic))

for pic in Listado_Test_Gris:
  Test_LBP_List.append(LBP(pic))

**¿Qué características tienen las imágenes procesadas con LBP?**

Una imagen procesada con LBP otorga una mejor visualizacion para los detalles mas finos y las texturas de la imagen, propiciando una mejor extraccion de caracteristicas. Por otra parte, tras este proceso los pixeles tendran una intensidad entre [0,255], lo que permite hacer un histograma de 256 bins

# **Desarrollo P2**

Extraer_Caract()

Dado una imagen de 500x600, devuelve un vector con histogramas resultantes de divivdir la imagen en 30 regiones

In [9]:
def Extraer_Caract(Imagen, Normalizar=False):
  Hist_Arr = np.zeros((0,))
  # se recorre la matriz de la imagen para dividir en 30 zonas
  for row in range(0, Imagen.shape[0], 100): 
    for col in range(0, Imagen.shape[1], 100):
      # se calcula el histograma de la zona actual
      hist, bin_edges = np.histogram(Imagen[row:row+100, col:col+100], bins=59, density=Normalizar)
      Hist_Arr = np.concatenate((Hist_Arr, hist))
  return Hist_Arr

# **Desarrollo P3**

distancia()

A partir de dos vectores de largo n, calcula la distancia entre ellos de la forma otorgada en el enunciado

In [10]:
def distancia(vec1, vec2):
  assert len(vec1) == len(vec2)
  n = len(vec1)
  dist = 0
  for i in range(0, n):
    dist = dist + abs(vec1[i] - vec2[i])
  return dist

# **Desarrollo P4**

Utilizando la variable *Gal_LBP_List* que contiene la transformacion LBP de las imagenes del archivo *ojos_gal.zip*, se construira una base de datos.



In [11]:
DB_Gal = []
# se recorre la lista de las imagenes de gal, para calcular sus caracteristicas
for matriz in Gal_LBP_List:
  Gal_caract = Extraer_Caract(matriz, True)
  # se guarda la extraccion caracteristicas de las imagenes
  DB_Gal.append(Gal_caract)

# **Desarrollo P5**

reconocimiento() determina si las imagenes otorgadas en una lista coinciden con las que se encuentran en un 'database'

In [12]:
def reconocimiento(DB, lista, umbral):
  Lista_minimos = []
  Sujeto_Base_Datos = 1
  for matriz in lista:
    Sujeto_a_comparar = 0
    Lista_Comparacion = []
    for caract in DB:      
      test_caract = Extraer_Caract(matriz, True)
      Distancia_Elementos = distancia(caract, test_caract)
      Lista_Comparacion.append(Distancia_Elementos)
      Sujeto_a_comparar += 1
    if min(Lista_Comparacion) < umbral:
      Lista_minimos.append(min(Lista_Comparacion))
      print('El sujeto del intento ' + str(Lista_Comparacion.index(min(Lista_Comparacion)) + 1) +
          ' ha sido identificado como el sujeto ' + str(Sujeto_Base_Datos) +
          ' en nuestra base de datos. Distancia mínima: ' + str(min(Lista_Comparacion)))
    else:
      print('No se ha encontrado el sujeto en la base de datos.')
    Sujeto_Base_Datos += 1
  print('Sujetos encontrados en la base de datos: ' + str(len(Lista_minimos)) + '/' + str(len(lista)))

Prueba de reconocimiento entre las imagenes del los archivos *ojos_gal.zip* y *ojos_test.zip*

In [13]:
reconocimiento(DB_Gal, Test_LBP_List, umbral=23)

El sujeto del intento 1 ha sido identificado como el sujeto 1 en nuestra base de datos. Distancia mínima: 21.321692307692295
El sujeto del intento 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 7.099061538461538
El sujeto del intento 3 ha sido identificado como el sujeto 3 en nuestra base de datos. Distancia mínima: 11.92163076923077
El sujeto del intento 4 ha sido identificado como el sujeto 4 en nuestra base de datos. Distancia mínima: 16.29943076923076
El sujeto del intento 5 ha sido identificado como el sujeto 5 en nuestra base de datos. Distancia mínima: 18.337199999999992
El sujeto del intento 6 ha sido identificado como el sujeto 6 en nuestra base de datos. Distancia mínima: 22.27295384615384
El sujeto del intento 7 ha sido identificado como el sujeto 7 en nuestra base de datos. Distancia mínima: 18.628569230769234
El sujeto del intento 8 ha sido identificado como el sujeto 8 en nuestra base de datos. Distancia mínima: 21.861769230769205
El s

Con lo anterior se determina que se realizan 10 reconocimientos correctos

# Desarrollo P6

Para el caso que se tenga una entrada una imagen de ojo de un individuo que no está en la base de datos, como las imagenes presentes en le archivo *ojos_impostor.rar*, se transformaria esta imagen a una escala de grises, para luego ecualizarla y asi distribuir de una forma mas uniforme su histograma. Luego aplicar la transformación LBP, para utilizar la funcion reconocimiento(), la cual determina si la imagen esta en la base de datos al calcular un vector de caracteristicas, al comparar la distancia minima entre estos vectores y un umbral.



In [14]:
reconocimiento(DB_Gal, Impostor_LBP_List, umbral=23)

No se ha encontrado el sujeto en la base de datos.
El sujeto del intento 2 ha sido identificado como el sujeto 2 en nuestra base de datos. Distancia mínima: 13.113430769230773
No se ha encontrado el sujeto en la base de datos.
Sujetos encontrados en la base de datos: 1/3


Con lo anterior se determina que el primer y tercer ojo corresponderian a impostores